In [1]:
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from spektral.datasets import qm9
from spektral.layers import EdgeConditionedConv, GlobalAttentionPool
from spektral.utils import label_to_one_hot

import psi4
from keras.models import load_model

Using TensorFlow backend.
/Users/em/anaconda3/envs/venv/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
def get_molecule_from_file(filenum):
    # Assumes dataset is in sibling directory
    f = open("/Users/em/2019-2020/Syslab/dsgdb9nsd.xyz/dsgdb9nsd_" + str(filenum).zfill(6) + ".xyz", "r")
    lines = f.readlines()
    f.close()
    num_atoms = int(lines[0])
    atom_list = lines[2:2+num_atoms]
    for i in range(len(atom_list)):
        atom_list[i] = atom_list[i][:atom_list[i].rfind("\t")] + "\n"
    return psi4.geometry("".join(atom_list))

In [3]:
testm = get_molecule_from_file(1)

In [4]:
X_in = Input(shape=(8, 5))
A_in = Input(shape=(8, 8))
E_in = Input(shape=(8, 8, 1))

W1121 10:36:58.611440 4446553536 deprecation_wrapper.py:119] From /Users/em/anaconda3/envs/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1121 10:36:58.631353 4446553536 deprecation_wrapper.py:119] From /Users/em/anaconda3/envs/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [5]:
gc1 = EdgeConditionedConv(64, activation='relu')([X_in, A_in, E_in])
gc2 = EdgeConditionedConv(128, activation='relu')([gc1, A_in, E_in])
pool = GlobalAttentionPool(256)(gc2)
dense1 = Dense(256, activation='relu')(pool)

W1121 10:36:59.200768 4446553536 deprecation_wrapper.py:119] From /Users/em/anaconda3/envs/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [6]:
output = Dense(1)(dense1)

In [7]:
model = Model(inputs=[X_in, A_in, E_in], outputs=output)
optimizer = Adam(lr=1e-3)
model.compile(optimizer=optimizer, loss='mse')
model.summary()

W1121 10:37:00.228387 4446553536 deprecation_wrapper.py:119] From /Users/em/anaconda3/envs/venv/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 8, 5)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 8, 8)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 8, 8, 1)      0                                            
__________________________________________________________________________________________________
edge_conditioned_conv_1 (EdgeCo (None, 8, 64)        704         input_1[0][0]                    
                                                                 input_2[0][0]              

In [8]:
model.load_weights('my_model_weights.h5')

W1121 10:37:00.761236 4446553536 deprecation_wrapper.py:119] From /Users/em/anaconda3/envs/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1121 10:37:00.761925 4446553536 deprecation_wrapper.py:119] From /Users/em/anaconda3/envs/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [9]:
preds = model.predict([X_test, A_test, E_test])

NameError: name 'X_test' is not defined